In [1]:
# Standard imports
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline
import pandas as pd
import plotly.express as px
import plotly.io as pio; pio.renderers.default='notebook'
import pickle

# MSLB imports
from mslb.slmanager import *
from mslb.slbatchselector import *
from mslb.slprocessor import *
from mslb.slmetric import *
from mslb.slworkflow import *
from mslb.slinitializer import *

# SMLB imports
from smlb.learners.scikit_learn.random_forest_regression_sklearn import RandomForestRegressionSklearn
from smlb.learners.lolo.random_forest_regression_lolo import RandomForestRegressionLolo
from smlb.learners.scikit_learn.gaussian_process_regression_sklearn import GaussianProcessRegressionSklearn
from smlb.features.matminer_composition import MatminerCompositionFeatures
from smlb.core.metrics import RootMeanSquaredError, MeanLogPredictiveDensity
from smlb import IdentityFeatures
from smlb.core.metrics import UncertaintyCorrelation, StandardConfidence, RootMeanSquareStandardizedResiduals

/Users/jkoeller/anaconda3/envs/citrine/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from smlb.datasets.experimental.band_gaps_sc73.band_gaps_sc73 import \
    BandGapsStrehlowCook1973Dataset 
SC_dataset = BandGapsStrehlowCook1973Dataset(filter_='bg', join=1, samplef=lambda e: e['formula'], labelf=np.median)

In [3]:
import smlb
features = MatminerCompositionFeatures(ionic_fast=True)
featurized_data = features.apply(SC_dataset)

training_sampler = smlb.RandomSubsetSampler(size=50, rng=0)
training_data = training_sampler.apply(featurized_data)
test_data = smlb.core.data.complement(featurized_data, training_data)

# pred_dist = learner.fit(training_data).apply(test_data)
# true_dist = smlb.core.distributions.DeltaPredictiveDistribution(test_data.labels())

/Users/jkoeller/anaconda3/envs/citrine/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.neighbors.unsupervised module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.



In [4]:
# New learners for each
forest1 = RandomForestRegressionSklearn(rng=0)
forest1.fit(training_data)
foo = forest1.apply(test_data)
forest2 = RandomForestRegressionSklearn(rng=0)
forest2.fit(training_data)
bar = forest2.apply(test_data)
print(list(zip(foo.mean[:10], bar.mean[:10])))

[(7.545850000000005, 7.545850000000005), (6.318405000000001, 6.318405000000001), (2.4127050000000003, 2.4127050000000003), (6.366800000000001, 6.366800000000001), (2.869305000000001, 2.869305000000001), (4.733239999999999, 4.733239999999999), (4.5072399999999995, 4.5072399999999995), (3.9083799999999997, 3.9083799999999997), (2.1522999999999985, 2.1522999999999985), (3.8890549999999995, 3.8890549999999995)]


In [5]:
# Repeat use of one learner
forest1 = RandomForestRegressionSklearn(rng=0)
forest1.fit(training_data)
foo = forest1.apply(test_data)
forest1.fit(training_data)
bar = forest1.apply(test_data)
print(list(zip(foo.mean[:10], bar.mean[:10])))

[(7.545850000000005, 7.501000000000006), (6.318405000000001, 6.343540000000003), (2.4127050000000003, 2.6650099999999997), (6.366800000000001, 6.313450000000002), (2.869305000000001, 3.073225), (4.733239999999999, 4.724149999999999), (4.5072399999999995, 4.476249999999999), (3.9083799999999997, 4.088600000000001), (2.1522999999999985, 2.1067849999999995), (3.8890549999999995, 3.9162049999999997)]


In [6]:
# Try copying an existing learner
from copy import copy
foresta = copy(forest1)
foresta.fit(training_data)
foo = foresta.apply(test_data)
forestb = copy(forest1)
forestb.fit(training_data)
bar = forestb.apply(test_data)
print(list(zip(foo.mean[:10], bar.mean[:10])))

[(7.282900000000004, 7.463000000000002), (6.317700000000001, 6.930750000000002), (2.432973333333334, 2.523249999999999), (6.149449999999999, 6.834450000000001), (2.545395000000001, 2.7070900000000004), (4.595900000000001, 4.777899999999999), (4.448900000000002, 4.207655000000002), (3.970290000000001, 4.277440000000001), (2.024645, 2.104615), (3.5815499999999996, 3.955234999999999)]


In [12]:
# Deepcopy an existing learner
from copy import deepcopy
forest1 = RandomForestRegressionSklearn(rng=0)
forest1.fit(training_data)
foo = forest1.apply(test_data)
forest2 = deepcopy(forest1)
forest2.fit(training_data)
bar = forest2.apply(test_data)
print(list(zip(foo.mean[:10], bar.mean[:10])))

RecursionError: maximum recursion depth exceeded while calling a Python object

In [13]:
# Sklearn clone doesn't work
from sklearn.base import clone
forest1 = RandomForestRegressionSklearn(rng=0)
forest1.fit(training_data)
foo = forest1.apply(test_data)
forest2 = clone(forest1)
forest2.fit(training_data)
bar = forest2.apply(test_data)
print(list(zip(foo.mean[:10], bar.mean[:10])))

TypeError: Cannot clone object '<smlb.learners.scikit_learn.random_forest_regression_sklearn.RandomForestRegressionSklearn object at 0x7fb81353d590>' (type <class 'smlb.learners.scikit_learn.random_forest_regression_sklearn.RandomForestRegressionSklearn'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' methods.